In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


In [3]:
!pip install torch_geometric

In [4]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


# Requirements

In [5]:
import torch

from base import OptimizerConfig, cross_validation
from base import SimplePytorchData, SimplePytorchDataTrainTestSplit
from base import SimpleTrainer, SimpleTester
from src.config import Node2VecConfig, Node2VecOptimizerConfig, SimpleClassifierConfig, GraphAutoEncoderConfig
from src.features import get_associations, get_homogeneous_graph, get_gae_pair_embedd_for_training_data
from src.models import SimpleMDAClassifier, SimpleMDAClassifierFactory
from src.utils import train_test_sampler, prj_logger
from torch_geometric.nn import GCNConv

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:

import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

In [8]:
logger = prj_logger.getLogger(__name__)

# Test

In [9]:
from torch_geometric.utils import to_networkx
import networkx as nx

from sklearn.decomposition import PCA
import numpy as np

In [10]:
homo = get_homogeneous_graph('cpu')

homo

2024-02-11 14:38:24,322 [INFO] Calling get_homogeneous_graph
2024-02-11 14:38:26,555 [INFO] homogeneous data : Data(x=[66911, 1], edge_index=[2, 633662])


Data(x=[66911, 1], edge_index=[2, 633662])

In [15]:
xgraph = to_networkx(homo)
adj = nx.adjacency_matrix(xgraph).todense()

In [12]:
adj = torch.from_numpy(adj).to_sparse()

In [17]:
adj

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
interaction = adj
dot_product = interaction @ interaction.T

sum_rows = np.sum(interaction, axis=1)
logger.info(f'sum rows :\n{sum_rows[:5]}')
union = np.expand_dims(sum_rows, axis=1) + np.expand_dims(sum_rows, axis=0) - dot_product
logger.info(f'union :\n{union[:5, :5]}')
similarity_matrix = dot_product / union
similarity_matrix = np.nan_to_num(similarity_matrix)
logger.info(f'similarity matrix :\n{similarity_matrix[:5, :5]}')

RuntimeError: "sparse_matmul" not implemented for 'Long'